In [1]:
import numpy as np
import pandas as pd
from sklearn import tree,linear_model,neural_network
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
import re

C:\Users\mbsaifee\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train= pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
predictions = pd.read_csv("gender_submission.csv")

In [3]:
labels=train.Survived
train.drop(columns=['Survived'],inplace=True)

In [4]:
join=pd.concat([train,test])

In [5]:
join.drop(columns=['PassengerId','Cabin','Fare','Ticket'],inplace=True)

In [6]:
join.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         263
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [7]:
Sex = pd.get_dummies(join.Sex)
Sex = Sex.male
join.Sex = Sex

In [8]:
Embarked = pd.get_dummies(join.Embarked)
Embarked = Embarked.iloc[:,0:2]
join = pd.concat([join,Embarked],axis = 1)
join.drop(['Embarked'],axis = 1,inplace = True)

In [9]:
join.isnull().sum()

Pclass      0
Name        0
Sex         0
Age       263
SibSp       0
Parch       0
C           0
Q           0
dtype: int64

In [10]:
pclass=pd.get_dummies(join.Pclass)
pclass=pclass.iloc[:,0:2]
join=pd.concat([join,pclass],axis=1)
join.drop(columns=['Pclass'],inplace=True)

In [11]:
lis=[]
for i in join.Name:
    start = re.escape(", ")
    end   = re.escape(". ")
    result = re.search('%s(.*)%s' % (start, end), i).group(1)
    lis.append(result)

join.Name=lis

In [12]:
join.Name.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Mrs. Martin (Elizabeth L', 'Lady', 'Sir', 'Mlle', 'Col',
       'Capt', 'the Countess', 'Jonkheer', 'Dona'], dtype=object)

In [13]:
for i in range(len(join)):
    if join.Name.values[i]=='Don' or join.Name.values[i]=='Dr' or join.Name.values[i]=='Major' or join.Name.values[i]=='Rev' or join.Name.values[i]=='Col' or join.Name.values[i]=='Capt' or join.Name.values[i]=='Dona' or join.Name.values[i]=='Jonkheer' or join.Name.values[i]=='the Countess' or join.Name.values[i]=='Sir':
        join.Name.values[i]='Mr'
    if join.Name.values[i]=='Lady' or join.Name.values[i]=='Mlle' or join.Name.values[i]=='Ms' or join.Name.values[i]=='Mme' or join.Name.values[i]=='Mrs. Martin (Elizabeth L':
        join.Name.values[i]='Mrs'

In [14]:
join.Name.unique()

array(['Mr', 'Mrs', 'Miss', 'Master'], dtype=object)

In [15]:
dic={'Mr':1,'Mrs':2,'Miss':3,'Master':4}
join.Name=join.Name.map(dic)

In [16]:
dic1={1:join.groupby(['Name']).Age.mean()[1],2:join.groupby(['Name']).Age.mean()[2],3:join.groupby(['Name']).Age.mean()[3],4:join.groupby(['Name']).Age.mean()[4],}


In [17]:
join.Age.fillna(value=join.Name.map(dic1),inplace=True)

In [18]:
join.isnull().sum()

Name     0
Sex      0
Age      0
SibSp    0
Parch    0
C        0
Q        0
1        0
2        0
dtype: int64

In [19]:
join['Family']=0
for i in range(len(join)):
    join.Family.values[i]=join.SibSp.values[i]+join.Parch.values[i]

In [20]:
for i in range(len(join)):
    if join.Family.values[i]>0:
        join.Family.values[i]=1

In [21]:
join.Family.unique()

array([1, 0], dtype=int64)

In [22]:
join.isnull().sum()

Name      0
Sex       0
Age       0
SibSp     0
Parch     0
C         0
Q         0
1         0
2         0
Family    0
dtype: int64

In [23]:
join.drop(columns=['Name'],inplace=True)

In [24]:
join.corr()

,Sex,Age,SibSp,Parch,C,Q,1,2,Family
Sex,1.000000,0.086519,-0.109609,-0.213125,-0.066564,-0.088651,-0.107371,-0.028862,-0.284537
Age,0.086519,1.000000,-0.221898,-0.140485,0.081809,-0.044880,0.369295,-0.013146,-0.133672
SibSp,-0.109609,-0.221898,1.000000,0.373587,-0.048396,-0.048678,-0.034256,-0.052419,0.591077
Parch,-0.213125,-0.140485,0.373587,1.000000,-0.008635,-0.100943,-0.013033,-0.010057,0.549022
C,-0.066564,0.081809,-0.048396,-0.008635,1.000000,-0.164166,0.325722,-0.134675,0.107874
Q,-0.088651,-0.044880,-0.048678,-0.100943,-0.164166,1.000000,-0.166101,-0.121973,-0.127214
1,-0.107371,0.369295,-0.034256,-0.013033,0.325722,-0.166101,1.000000,-0.296526,0.126551
2,-0.028862,-0.013146,-0.052419,-0.010057,-0.134675,-0.121973,-0.296526,1.000000,0.035075
Family,-0.284537,-0.133672,0.591077,0.549022,0.107874,-0.127214,0.126551,0.035075,1.000000


In [25]:
x_test=join.iloc[891: :, :]
x_cv=join.iloc[801:891:, :]
x_train=join.iloc[ :801:, :]

In [26]:
y_cv=labels.iloc[801:]
y_train=labels.iloc[:801]

In [27]:
clf = tree.DecisionTreeClassifier(criterion="entropy",)
clf.fit(x_train, y_train)
train_accuracy = clf.score(x_cv,y_cv)

In [28]:
train_accuracy

0.8111111111111111

In [29]:
y_pred=clf.predict(x_test)

In [30]:
clf1=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(18), random_state=0)
clf1.fit(x_train,y_train)
train_accuracy1=clf1.score(x_cv,y_cv)
train_accuracy1

0.8888888888888888

In [31]:
y_pred1=clf1.predict(x_test)


In [32]:
clf2= LogisticRegression(C=1000.0, random_state=1)
clf2.fit(x_train,y_train)
train_accuracy2=clf2.score(x_cv,y_cv)
train_accuracy2

0.8444444444444444

In [33]:
y_pred2=clf2.predict(x_test)

In [34]:
clf3= AdaBoostClassifier(n_estimators=100)
clf3.fit(x_train,y_train)
train_accuracy3=clf3.score(x_cv,y_cv)
train_accuracy3

0.8333333333333334

In [35]:
y_pred3=clf3.predict(x_test)

In [36]:
clf4=RandomForestClassifier(n_estimators=800, max_features=9)
clf4.fit(x_train,y_train)
train_accuracy4=clf4.score(x_cv,y_cv)
train_accuracy4

0.8222222222222222

In [37]:
y_pred4=clf4.predict(x_test)

In [38]:
clf5 = BaggingClassifier(base_estimator=clf1, n_estimators=100, random_state=7)
clf5.fit(x_train,y_train)
train_accuracy5=clf5.score(x_cv,y_cv)
train_accuracy5

0.8666666666666667

In [39]:
y_pred5=clf5.predict(x_test)

In [40]:
y=clf.predict(x_train)
y1=clf1.predict(x_train)
y2=clf2.predict(x_train)
y3=clf3.predict(x_train)
y4=clf4.predict(x_train)
y5=clf5.predict(x_train)

In [41]:
Xy=np.vstack([y,y1,y2,y3,y4,y5]).T

In [42]:
clf6 = GradientBoostingClassifier()
clf6.fit(Xy, y_train)
clf6.score(Xy,y_train)

0.9375780274656679

In [43]:
Xy_test=np.vstack([y_pred,y_pred1,y_pred2,y_pred3,y_pred4,y_pred5]).T

In [44]:
y_pred_final=clf6.predict(Xy_test)
y_pred_final

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [45]:
predictions['Survived']=y_pred_final

In [46]:
predictions.to_csv("pred.csv",index = False)